#### The regression model did not provide good accuracy, so we can consider using a boosting algorithm.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.feature_selection import chi2

import warnings
warnings.filterwarnings('ignore')


In [2]:
#!pip install catboost

In [3]:
data=pd.read_csv("autism.csv")

In [4]:
data.head()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,jaundice,austim,Class/ASD
0,1,0,1,1,1,1,0,1,1,1,18.605397,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,13.829369,0,0,0,0
2,1,1,1,1,1,1,0,0,1,1,14.679893,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,61.035288,0,0,0,0
4,0,0,0,0,1,0,0,0,1,1,14.256686,1,0,1,0


In [5]:
df=data

In [6]:
def selectkbest(indep_X,dep_Y,n):
        test = SelectKBest(score_func=chi2, k=n)
        fit1= test.fit(indep_X,dep_Y)
        selectk_features = fit1.transform(indep_X)
        return selectk_features

In [7]:
def split_scalar(indep_x,dep_y):
    X_train,X_test,y_train,y_test=train_test_split(indep_x,dep_y,test_size=0.25,random_state=0)
    sc=StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test=sc.transform(X_test)
    return X_train,X_test,y_train,y_test

In [8]:
def cm_prediction(classifier,X_test):
     y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
     from sklearn.metrics import confusion_matrix
     cm = confusion_matrix(y_test, y_pred)
        
     from sklearn.metrics import accuracy_score 
     from sklearn.metrics import classification_report 
        #from sklearn.metrics import confusion_matrix
        #cm = confusion_matrix(y_test, y_pred)
        
     Accuracy=accuracy_score(y_test, y_pred )
        
     report=classification_report(y_test, y_pred)
     return  classifier,Accuracy,report,X_test,y_test,cm

In [9]:
def gradientboost(X_train,y_train,X_test,y_test):
    gb = GradientBoostingClassifier(random_state=42)
    param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'subsample': [0.8, 1.0]
        }
    grid= GridSearchCV(estimator=gb, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
    grid.fit(X_train, y_train)
    classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(grid,X_test)
    return  classifier,Accuracy,report,X_test,y_test,cm 
    

In [10]:
def xgboost(X_train,y_train,X_test,y_test):
    xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
    param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'gamma': [0, 0.1, 0.2]
    }
    grid = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)

    # Fit the model to the training data
    grid.fit(X_train, y_train)
    classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(grid,X_test)
    return  classifier,Accuracy,report,X_test,y_test,cm 
    
    

In [11]:
def catboost(X_train,y_train,X_test,y_test):
    catboost = CatBoostClassifier(random_state=42, silent=True)
    param_grid = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [3, 5, 7, 9],
    'l2_leaf_reg': [1, 3, 5, 7],
    'border_count': [32, 64, 128],
    'bagging_temperature': [0.0, 0.5, 1.0]
    }
    grid = GridSearchCV(estimator=catboost, param_grid=param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
    grid.fit(X_train, y_train)
    classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(grid,X_test)
    return  classifier,Accuracy,report,X_test,y_test,cm 
    

In [12]:
def selectk_Classification(accgradient,accxgboost,acccatboost): 
    
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['GradientBoosting','XGBoost','CatBoost'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['GradientBoosting'][idex]=accgradient[number]       
        dataframe['XGBoost'][idex]=accxgboost[number]
        dataframe['CatBoost'][idex]=acccatboost[number]
        
    return dataframe

In [13]:
indep_X=df.drop(['Class/ASD'], axis=1)
dep_Y=df['Class/ASD']

In [14]:
kbest=selectkbest(indep_X,dep_Y,3)   

accgradient=[]
accxgboost=[]
acccatboost=[]


In [15]:
X_train,X_test,y_train,y_test = train_test_split(indep_X,dep_Y,test_size=0.30,random_state=0)

In [16]:
classifier,Accuracy,report,X_test,y_test,cm=gradientboost(X_train,y_train,X_test,y_test)
accgradient.append(Accuracy)

classifier,Accuracy,report,X_test,y_test,cm=xgboost(X_train,y_train,X_test,y_test)  
accxgboost.append(Accuracy)
    
classifier,Accuracy,report,X_test,y_test,cm=catboost(X_train,y_train,X_test,y_test)  
acccatboost.append(Accuracy)

result=selectk_Classification(accgradient,accxgboost,acccatboost)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits
Fitting 3 folds for each of 972 candidates, totalling 2916 fits
Fitting 3 folds for each of 1296 candidates, totalling 3888 fits


In [18]:
result

,GradientBoosting,XGBoost,CatBoost
ChiSquare,0.8625,0.870833,0.858333
